# Install dependencies

In [10]:
!pip -q install fastapi uvicorn scikit-learn joblib nest_asyncio pyngrok pandas



# Imports & dataset load

In [13]:
import pandas as pd
import numpy as np

# Load your uploaded dataset
df = pd.read_csv("/content/Iris.csv")   # since you uploaded Iris.csv

print("Shape:", df.shape)
print(df.head())



Shape: (150, 6)
   Id  SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm      Species
0   1            5.1           3.5            1.4           0.2  Iris-setosa
1   2            4.9           3.0            1.4           0.2  Iris-setosa
2   3            4.7           3.2            1.3           0.2  Iris-setosa
3   4            4.6           3.1            1.5           0.2  Iris-setosa
4   5            5.0           3.6            1.4           0.2  Iris-setosa


# Prepare features & target


In [14]:
# Columns: Id, SepalLengthCm, SepalWidthCm, PetalLengthCm, PetalWidthCm, Species
df = df.drop(columns=["Id"])

X = df.drop(columns=["Species"])
y = df["Species"]

print("Features:", X.columns.tolist())
print("Target classes:", y.unique())



Features: ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']
Target classes: ['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']


# Train/test split

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)


# Build pipeline & train

In [19]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(max_iter=200))
])

pipe.fit(X_train, y_train)
print("✅ Model trained")



✅ Model trained


# Evaluate model

In [20]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred = pipe.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.4f}")
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy: 0.9333

Classification Report:
                  precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        10
Iris-versicolor       0.90      0.90      0.90        10
 Iris-virginica       0.90      0.90      0.90        10

       accuracy                           0.93        30
      macro avg       0.93      0.93      0.93        30
   weighted avg       0.93      0.93      0.93        30


Confusion Matrix:
 [[10  0  0]
 [ 0  9  1]
 [ 0  1  9]]


# Save model & metadata

In [21]:
import joblib, json

joblib.dump(pipe, "model.pkl")

metadata = {
    "model_type": "Logistic Regression + StandardScaler",
    "problem_type": "classification",
    "features": X.columns.tolist(),
    "classes": sorted(y.unique().tolist()),
    "test_accuracy": float(acc)
}
with open("model_metadata.json", "w") as f:
    json.dump(metadata, f, indent=2)

print("✅ Saved model.pkl and model_metadata.json")


✅ Saved model.pkl and model_metadata.json


# Quick test prediction

In [23]:
sample = X_test.iloc[0].values.reshape(1, -1)
sample_df = pd.DataFrame(sample, columns=X_test.columns) # Convert to DataFrame with column names
pred = pipe.predict(sample_df)[0]
proba = pipe.predict_proba(sample_df).max()

print("Sample features:", dict(zip(X.columns, X_test.iloc[0])))
print("Prediction:", pred, "| Confidence:", round(proba, 3))

Sample features: {'SepalLengthCm': 4.4, 'SepalWidthCm': 3.0, 'PetalLengthCm': 1.3, 'PetalWidthCm': 0.2}
Prediction: Iris-setosa | Confidence: 0.979


# FastAPI app

In [28]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel

# Load model & metadata
model = joblib.load("model.pkl")
with open("model_metadata.json", "r") as f:
    MODEL_META = json.load(f)

app = FastAPI(title="Iris Classifier API (CSV version)")

class IrisInput(BaseModel):
    SepalLengthCm: float
    SepalWidthCm: float
    PetalLengthCm: float
    PetalWidthCm: float

class PredictionOutput(BaseModel):
    prediction: str
    confidence: float

@app.get("/")
def health_check():
    return {"status": "healthy", "message": "Iris Classifier API is running"}

@app.get("/model-info")
def model_info():
    return MODEL_META

@app.post("/predict", response_model=PredictionOutput)
def predict(inp: IrisInput):
    try:
        features = np.array([[inp.SepalLengthCm, inp.SepalWidthCm, inp.PetalLengthCm, inp.PetalWidthCm]])
        pred = model.predict(features)[0]
        proba = model.predict_proba(features).max()
        return {"prediction": pred, "confidence": float(proba)}
    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))
